In [8]:
import pandas as pd

# 读取文件，跳过以#开头的行（注释行），并用制表符\t分隔列
prot_info = pd.read_csv('./9606_prot_link/9606.protein.info.v12.0.txt', sep='\t', comment='#')
# 修正列名（如果第一行被误读为注释）
prot_info.columns = ['string_protein_id', 'preferred_name', 'protein_size', 'annotation']
# 显示前几行数据
print(prot_info.head())

      string_protein_id preferred_name  protein_size  \
0  9606.ENSP00000000412           M6PR           277   
1  9606.ENSP00000001008          FKBP4           459   
2  9606.ENSP00000001146        CYP26B1           512   
3  9606.ENSP00000002125        NDUFAF7           441   
4  9606.ENSP00000002165          FUCA2           467   

                                          annotation  
0  Cation-dependent mannose-6-phosphate receptor;...  
1  Peptidyl-prolyl cis-trans isomerase FKBP4, N-t...  
2  Cytochrome P450 26B1; Involved in the metaboli...  
3  Protein arginine methyltransferase NDUFAF7, mi...  
4  Plasma alpha-L-fucosidase; Alpha-L-fucosidase ...  


In [9]:
import pandas as pd

# 读取整个CSV文件
sl_data = pd.read_csv('./data/SLKB_rawSL.csv')

# 查看前5行
print(sl_data.head())


   Unnamed: 0     gene_pair  study_origin cell_line_origin gene_1  gene_2  \
0           1   AKT1|AMBRA1      33956155             RPE1   AKT1  AMBRA1   
1           2   AKT3|AMBRA1      33956155             RPE1   AKT3  AMBRA1   
2           3   AMBRA1|ARF6      33956155             RPE1   ARF6  AMBRA1   
3           4   AMBRA1|ATF4      33956155             RPE1   ATF4  AMBRA1   
4           5  AMBRA1|ATG10      33956155             RPE1  ATG10  AMBRA1   

  SL_or_not  SL_score  statistical_score  SL_score_cutoff  \
0    Not SL -0.010982                0.0             -1.0   
1    Not SL  2.159344                0.0             -1.0   
2    Not SL -0.564699                0.0             -1.0   
3    Not SL  0.999030                0.0             -1.0   
4    Not SL  3.916281                0.0             -1.0   

   statistical_score_cutoff  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                   

In [11]:
import pandas as pd

# 假设已有数据：
# sl_data - 包含 gene_1 和 gene_2 列
# prot_info - 包含 preferred_name 和 string_protein_id 列

# 步骤1：创建从 preferred_name 到 string_protein_id 的映射字典
name_to_protid = dict(zip(prot_info['preferred_name'], prot_info['string_protein_id']))

# 步骤2：为 sl_data 添加新列
sl_data['gene_1_protid'] = sl_data['gene_1'].map(name_to_protid)
sl_data['gene_2_protid'] = sl_data['gene_2'].map(name_to_protid)

# 步骤3：检查映射结果
print("添加蛋白质ID后的数据样例：")
print(sl_data[['gene_1', 'gene_2', 'gene_1_protid', 'gene_2_protid']].head())

# 步骤4：统计映射成功率
total_pairs = len(sl_data)
mapped_pairs = sl_data[sl_data['gene_1_protid'].notna() & sl_data['gene_2_protid'].notna()].shape[0]

print(f"\n映射统计：")
print(f"总基因对数：{total_pairs}")
print(f"完整映射的基因对数：{mapped_pairs} ({mapped_pairs/total_pairs:.1%})")
print(f"gene_1 映射成功率：{sl_data['gene_1_protid'].notna().mean():.1%}")
print(f"gene_2 映射成功率：{sl_data['gene_2_prot_id'].notna().mean():.1%}")

添加蛋白质ID后的数据样例：
  gene_1  gene_2         gene_1_protid         gene_2_protid
0   AKT1  AMBRA1  9606.ENSP00000451828  9606.ENSP00000431926
1   AKT3  AMBRA1  9606.ENSP00000500582  9606.ENSP00000431926
2   ARF6  AMBRA1  9606.ENSP00000298316  9606.ENSP00000431926
3   ATF4  AMBRA1  9606.ENSP00000336790  9606.ENSP00000431926
4  ATG10  AMBRA1  9606.ENSP00000282185  9606.ENSP00000431926

映射统计：
总基因对数：280483
完整映射的基因对数：254243 (90.6%)
gene_1 映射成功率：94.3%
gene_2 映射成功率：96.0%


In [12]:
import pandas as pd

# 计算覆盖率的函数
def calculate_coverage_stats(group):
    # 计算gene_1的覆盖率
    gene_1_total = group['gene_1'].nunique()
    gene_1_covered = group[group['gene_1_protid'].notnull()]['gene_1'].nunique()
    gene_1_coverage = gene_1_covered / gene_1_total if gene_1_total > 0 else 0
    
    # 计算gene_2的覆盖率
    gene_2_total = group['gene_2'].nunique()
    gene_2_covered = group[group['gene_2_protid'].notnull()]['gene_2'].nunique()
    gene_2_coverage = gene_2_covered / gene_2_total if gene_2_total > 0 else 0
    
    # 计算基因对覆盖率
    total_pairs = len(group)
    covered_pairs = group[group['gene_1_protid'].notnull() & group['gene_2_protid'].notnull()].shape[0]
    pair_coverage = covered_pairs / total_pairs if total_pairs > 0 else 0
    
    return pd.Series({
        'gene_1_unique_count': gene_1_total,
        'gene_1_covered_count': gene_1_covered,
        'gene_1_coverage': gene_1_coverage,
        'gene_2_unique_count': gene_2_total,
        'gene_2_covered_count': gene_2_covered,
        'gene_2_coverage': gene_2_coverage,
        'total_pairs': total_pairs,
        'covered_pairs': covered_pairs,
        'pair_coverage': pair_coverage
    })

# 按cell_line_origin分组计算覆盖率
coverage_by_cell_line = sl_data.groupby('cell_line_origin').apply(calculate_coverage_stats).reset_index()

# 格式化输出百分比
percentage_cols = ['gene_1_coverage', 'gene_2_coverage', 'pair_coverage']
for col in percentage_cols:
    coverage_by_cell_line[col] = coverage_by_cell_line[col].apply(lambda x: f"{x:.2%}")

# 显示结果
print("按cell_line_origin分组的覆盖率统计:")
print(coverage_by_cell_line)

# 可选：保存结果到CSV
# coverage_by_cell_line.to_csv('coverage_by_cell_line.csv', index=False)\
# 选择要显示的列
cols_to_show = ['cell_line_origin'] + percentage_cols
print(coverage_by_cell_line[cols_to_show])

按cell_line_origin分组的覆盖率统计:
   cell_line_origin  gene_1_unique_count  gene_1_covered_count  \
0             22RV1                 49.0                  49.0   
1              293T                 72.0                  71.0   
2              786O                 24.0                  24.0   
3              A375                923.0                 884.0   
4              A549               2160.0                2126.0   
5               GI1               2092.0                2060.0   
6              HELA               1140.0                1128.0   
7            HS936T               2092.0                2060.0   
8            HS944T               2092.0                2060.0   
9              HSC5               2092.0                2060.0   
10             HT29                 24.0                  24.0   
11           IPC298               2092.0                2060.0   
12           JURKAT                387.0                 360.0   
13             K562                644.0         

C:\Users\DELL\AppData\Local\Temp\ipykernel_12424\623208008.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  coverage_by_cell_line = sl_data.groupby('cell_line_origin').apply(calculate_coverage_stats).reset_index()
